### Script to stratify sampling of filtered data to be used for Manual Labeling

In [9]:
import os
import pandas as pd
import random
from datetime import datetime
import json

In [11]:
# Parameters

#Currently uses old data --> Change to new climate_labeled_posts from HPC 
INPUT_JSON = "/Users/tobiasmichelsen/Bachelor_Project/DS_BachelorProject_PH/data/processed/Climate_labeled_posts.json" 

OUTPUT_PATH = "../processed/manual_labels_climate.csv"

SAMPLE_SIZE = 500

In [18]:
with open(INPUT_JSON, "r", encoding="utf-8") as f:
    data = json.load(f)


def extract_hour(ts):
    try:
        return datetime.fromisoformat(ts.replace("Z", "+00:00")).hour
    except:
        return None

def length_bucket(text):
    l = len(text)
    if l < 100:
        return "short"
    elif l < 300:
        return "medium"
    else:
        return "long"

In [20]:
df = pd.DataFrame(data)

df["text_length"] = df["text"].apply(len)
df["length_bucket"] = df["text"].apply(length_bucket)
df["hour"] = df["timestamp"].apply(extract_hour)

df["hour"] = df["hour"].astype(int)

KeyError: 'timestamp'

In [17]:
#Proportional strafied sampling

strata = df.groupby("length_bucket")
total_count = len(df)
stratum_sizes = strata.size()
proportions = stratum_sizes / total_count
samples_per_stratum = (proportions * SAMPLE_SIZE).round().astype(int)

In [22]:
sampled_rows = []
for (group_key, group_df), n_samples in zip(strata, samples_per_stratum.items()):
    n = min(n_samples[1], len(group_df))
    if n > 0:
        sampled_rows.append(group_df.sample(n=n, random_state=42))

sample_df = pd.concat(sampled_rows, ignore_index=True)
sample_df = sample_df.sample(n=SAMPLE_SIZE, random_state=42)

In [23]:
#Write our label / annotation in extra field (Could be changed if we get biased by score but shouldnt matter)
sample_df["annotation"] = ""

In [24]:
half = SAMPLE_SIZE // 2

df_tobias = sample_df.iloc[:half].reset_index(drop=True)
df_abel = sample_df.iloc[half:].reset_index(drop=True)

In [ ]:
#Interested metadata for annotated file:
fields = ["repo", "seq", "text", "timestamp", "cid", "uri", "label", "score", "annotation"]

# One csv for each
tobias_data = df_tobias[fields].to_dict(orient="records")
friend_data = df_abel[fields].to_dict(orient="records")

In [25]:
with open(os.path.join(OUTPUT_PATH, "manual_labels_climate_Tobias.json"), "w", encoding="utf-8") as f:
    json.dump(tobias_data, f, indent=2, ensure_ascii=False)

with open(os.path.join(OUTPUT_PATH, "manual_labels_climate_Abel.json"), "w", encoding="utf-8") as f:
    json.dump(friend_data, f, indent=2, ensure_ascii=False)

FileNotFoundError: [Errno 2] No such file or directory: '../processed/manual_labels_climate.csv/manual_labels_climate_Tobias.json'